In [ ]:
import json
import numpy as np
import sklearn
import sklearn.metrics
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import datetime
# import snap
import random
import os
import playlist

In [ ]:
playlist_list = playlist.get_playlist_list()

In [ ]:

len(playlist_list)

In [ ]:
import sys
sys.getsizeof(playlist_list)

In [ ]:
def get_tracks(playlist):
    track_list = []
    for i in range(playlist['num_tracks']):
        track_list.append(playlist['tracks'][i]['track_uri'])
    return track_list

In [ ]:
def make_track_artist_dict(playlist_list):
    track_artist_dict = {}
    for playlist in playlist_list:
        for track in playlist['tracks']:
            track_artist_dict[track['track_uri']] = track['artist_uri']
    return track_artist_dict

track_artist_dict = make_track_artist_dict(playlist_list)

def get_track_uri_artist(track_uri):
    return track_artist_dict.get(track_uri)

In [ ]:
g, tracks_dict, tracks_id_dict = playlist.make_graph_dict(playlist_list)

In [ ]:
from collections import defaultdict
import random

def basic_random_walk_track_neighbors(query, g, dropped_track_id):
    N = 10000
    totalSteps = 0
    num_visits = defaultdict(int)
    
    while totalSteps < N:
        currTrack = query
        currSteps = 5
        for i in range(currSteps):
            # takes 1 step (from a track to playlist)
            edges = list(g[currPlaylist])
            if currPlaylist == query:
                edges.remove(dropped_track_id)
            currTrack = random.choice(edges)
            # takes 1 step (from a playlist to track)
            edges = list(g[currTrack])
            if currTrack == dropped_track_id:
                edges.remove(query)
            currTrack = random.choice(edges)
            num_visits[currTrack] += 1
        totalSteps += currSteps
    return num_visits

In [ ]:
def numbers_up_to(n):
    for i in range(n):
        yield i, i

In [ ]:
for k, j in numbers_up_to(4):
    print(k+j)

In [ ]:
# g, dropped_track_id,track_id_dict,tracks_dict  = make_graph_drop_edge(playlist_list, 20, 3)
# edges = list(g.GetNI(0).GetOutEdges())
# print edges
# print(dropped_track_id)

In [ ]:
def get_random_neighbors(graph, track_id, T):
    num_visit_dict = basic_random_walk_track_neighbors(track_id, graph)
    suggested_ids = sorted(num_visit_dict, key=num_visit_dict.get, reverse=True)
    if T < len(suggested_ids):
        return suggested_ids[:T]
    else:
        print 'The number of track neighbors are smaller than the amount asked'

In [ ]:
# result = get_random_neighbors(g, 2000, 5)
# print result
# print type(result)

In [ ]:
def get_random_negative_neighbors(graph, track_id, T, L, sample_num):
    if L < T and L > sample_num:
        top_t_suggested_ids = get_random_neighbors(graph, track_id, T)
        top_t_suggested_ids.reverse()
        negative_neighbors_list = np.random.choice(top_t_suggested_ids[:L], sample_num, replace=False)
    else:
        print 'error'
    return negative_neighbors_list
    

In [ ]:
# ran_result = get_random_negative_neighbors(g, 2000, 5, 3, 2)
# print ran_result

In [ ]:
def get_track_uri_from_track_id(given_track_id, track_id_dict):
    return track_id_dict[given_track_id]
    

In [ ]:
# res = get_track_uri_from_track_id(1970, track_id_dict)
# print res

In [ ]:
# MOVE THE CODE TO THIS FOLDER


track_w2vf_df = pd.read_csv("word2vecf-features.txt", skiprows=1, header=None, sep=' ')
track_w2vf_df = track_w2vf_df.set_index(0)
track_w2vf_df = track_w2vf_df.drop(columns=[track_w2vf_df.shape[1]])
track_w2vf_df.columns

In [ ]:
track_w2vf_df.shape

In [ ]:
track_w2vf_df.head()

In [ ]:
# ix = index
# track_df.ix['spotify:track:4eLSCSELtKxZwXnFbNLXT5']

In [ ]:
# import numpy as np
# np.array(track_w2vf_df.loc['spotify:track:4eLSCSELtKxZwXnFbNLXT5'])

In [ ]:
def track_id_to_vector(given_track_id, track_id_dict, track_df):
    track_uri = get_track_uri_from_track_id(given_track_id, track_id_dict)
    w2vf_vector = np.array(track_df.loc[track_uri])
    return w2vf_vector

In [ ]:
# vector = track_id_to_vector(2500, track_id_dict, track_w2vf_df)
# print vector

In [ ]:
def track_id_list_to_matrix(given_track_id_list, track_id_dict, track_df):
    vector_list = []
    for given_track_id in given_track_id_list:
        vector_list.append(track_id_to_vector(given_track_id, track_id_dict, track_df))
    return np.vstack(vector_list)

In [ ]:
# matrix = track_id_list_to_matrix([2000,2010,3000], track_id_dict, track_w2vf_df)
# print matrix.shape

In [ ]:
def make_track_dicts(playlist_list):
    tracks_dict = {}
    next_track_id = len(playlist_list)
    for playlist_id, playlist in enumerate(playlist_list):
        track_list = get_tracks(playlist)
        track_list = sorted(track_list)
        
        for track_index, track_uri in enumerate(track_list):
            if track_uri not in tracks_dict:
                tracks_dict[track_uri] = next_track_id
                next_track_id += 1
    track_id_dict = dict(zip(tracks_dict.values(), tracks_dict.keys()))
    return track_id_dict, tracks_dict

def get_dropped_track_id(track_id_dict, playlist_list, playlist_id, dropped_track_index):
    playlist = playlist_list[playlist_id]
    tracks = get_tracks(playlist)
    tracks.sort()
    track_uri = tracks[dropped_track_index]
    return tracks_dict[track_uri]

In [ ]:
track_id_dict, tracks_dict = make_track_dicts(playlist_list)

def generate_one_training_data(playlist_list, track_df):
    track_ids_in_playlist_list = []
    # 5 tries
    for i in range(5):
        playlist_id = random.randrange(len(playlist_list))
        dropped_track_index = random.randrange(playlist_list[playlist_id]['num_tracks'])
        dropped_track_id = get_dropped_track_id(track_id_dict, playlist_list, playlist_id, dropped_track_index)
    #         print dropped_track_id
    #         print track_id_dict
        dropped_track_uri = track_id_dict[dropped_track_id]
        if dropped_track_uri not in track_w2vf_df.index:
            continue
        track_vector = track_id_to_vector(dropped_track_id, track_id_dict, track_df)
        tracks = sorted(get_tracks(playlist_list[playlist_id]))
        for j, track_uri in enumerate(tracks):
            if j == dropped_track_index:
                continue
            if track_uri not in track_w2vf_df.index:
                continue
            track_ids_in_playlist_list.append(tracks_dict[track_uri])
        if not track_ids_in_playlist_list:
            continue
        query_matrix = track_id_list_to_matrix(track_ids_in_playlist_list, track_id_dict, track_df)
        query_vector = query_matrix.mean(0)
        return track_vector, query_vector
    return None, None

In [ ]:
from tqdm import tnrange

sample_row_number = len(tracks_dict)
track_vector_rows = np.empty((sample_row_number, 200))
query_vector_rows = np.empty((sample_row_number, 200))
for ii in tnrange(sample_row_number):
    track_vector, query_vector = generate_one_training_data(playlist_list, track_w2vf_df)
    if track_vector is None:
        continue
    track_vector_rows[ii,:] = track_vector
    query_vector_rows[ii,:] = query_vector

In [ ]:
print(track_vector_rows.shape)
print(query_vector_rows.shape)

In [ ]:
len(tracks_dict)

In [ ]:
np.savez_compressed('torch_query_track_vectors_%d_playlist_new' % (len(playlist_list),), track_vector_rows=track_vector_rows, query_vector_rows=query_vector_rows)

In [ ]:
# np.savez_compressed('torch_query_track_vectors_1k_playlist', track_vector_rows=track_vector_rows, query_vector_rows=query_vector_rows)

In [ ]:
# len(get_tracks(playlist_list[331]))

In [ ]:
# result_array = np.empty((0, 100))

# for line in data_array:
#     result = do_stuff(line)
#     result_array = np.append(result_array, [result], axis=0)

In [ ]:
track_vector, query_matrix = generate_one_training_data(playlist_list, track_w2vf_df)
print(track_vector.shape)
print(query_matrix.shape)

In [ ]:
# ! pip install torch

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.fc1 = nn.Linear(200, 50)
        self.fc2 = nn.Linear(50, 50)
        self.fc3 = nn.Linear(50, 200)

    def forward(self, vi):
        ui = self.fc1(vi)
        ui_rel = F.relu(ui)
        uui = F.relu(self.fc2(ui_rel))
        predictaed_vi = self.fc3(uui)
        return predictaed_vi

In [ ]:
from tqdm import tqdm_notebook as tqdm
from tqdm import tnrange

def train(model, optimizer, loader, num_epochs):
#     loader does the job of these 3 lines
#     track_vector, query_vector = generate_one_training_data(playlist_list, track_w2vf_df)
#     target = torch.from_numpy(track_vector).float()
#     data = torch.from_numpy(query_vector).float()
    model.train()
    losses = []
    total_steps = len(loader.dataset)/loader.batch_size
    for epoch in tnrange(num_epochs):
        for batch_idx, (data, target) in tqdm(enumerate(loader),total=total_steps,leave=False):
            optimizer.zero_grad()
            output = model(data)
            output_norm = output.norm(p=2, dim=-1)
            cosine_loss = F.mse_loss(output/output_norm.view(-1, 1),
                              target/target.norm(p=2, dim=-1).view(-1, 1))
            loss = cosine_loss + 0.001*output_norm.mean()

            loss.backward()
            optimizer.step()
            if (batch_idx % 100) == 0:
                losses.append(cosine_loss.item())
        print(np.mean(losses[-total_steps/100:]))
    return losses

In [ ]:
model = Net()
# optimizer = optim.SGD(model.parameters(), lr=5e-3, momentum=0.9)
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
train_data = np.load('torch_query_track_vectors_50000_playlist.npz')
# train_data = np.load('torch_query_track_vectors_100k_playlist.npz')
# data.keys()

In [ ]:
import torch.utils.data
# make it to torch dataset
dataset = torch.utils.data.TensorDataset(
    torch.from_numpy(train_data['query_vector_rows']).float(), 
    torch.from_numpy(train_data['track_vector_rows']).float())

In [ ]:
loader = torch.utils.data.DataLoader(dataset, batch_size = 32, shuffle = True)

In [ ]:
num_epochs = 30
losses = train(model, optimizer, loader, num_epochs)

In [ ]:
# avg_data_mse = np.mean((train_data['query_vector_rows']-train_data['track_vector_rows'])**2)
avg_data_cosine = np.mean((train_data['query_vector_rows']/np.linalg.norm(train_data['query_vector_rows'], axis=-1).reshape(-1, 1)
                           -
                           train_data['track_vector_rows']/np.linalg.norm(train_data['track_vector_rows'], axis=-1).reshape(-1, 1))**2)

In [ ]:
torch.save(model.state_dict(), 'neural_network.torch')

In [ ]:
model.load_state_dict(torch.load('neural_network.torch'))

In [ ]:
# import scipy.spatial.distance
# from tqdm import tqdm_notebook as tqdm
# from tqdm import tnrange
# a = train_data['query_vector_rows']
# b = train_data['track_vector_rows']
# num = a.shape[0]
# s = 0.0
# for i in tnrange(num):
#     s += scipy.spatial.distance.cosine(a[i,:], b[i,:])
# s/num

In [ ]:
train_data['query_vector_rows'][0,:].shape

In [ ]:
# print(avg_data_mse)
print(avg_data_cosine)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(losses, label='training loss')
plt.plot([avg_data_cosine]*len(losses), label='original cosine similarity')
plt.legend()
plt.show()

In [ ]:
model.fc3.weight

In [ ]:
w = model.fc1.weight.detach().numpy()
b = model.fc1.bias.detach().numpy()

In [ ]:
w2 = model.fc2.weight.detach().numpy()
b2 = model.fc2.bias.detach().numpy()

In [ ]:
proj_query_vector_rows = np.maximum(0, np.dot(train_data['query_vector_rows'], w.T) + b)
proj_track_vector_rows = np.maximum(0, np.dot(train_data['track_vector_rows'], w.T) + b)
proj_query_vector_rows = np.dot(proj_query_vector_rows, w2.T) + b2
proj_track_vector_rows = np.dot(proj_track_vector_rows, w2.T) + b2

a = proj_query_vector_rows
b = proj_track_vector_rows
avg_data_cosine = np.mean((a/np.linalg.norm(a, axis=-1).reshape(-1, 1)
                           -
                           b/np.linalg.norm(b, axis=-1).reshape(-1, 1))**2)
print(avg_data_cosine)

In [ ]:
tracks_dict, tracks_id_dict = playlist.make_tracks_dict(playlist_list)

In [ ]:
track_df = track_w2vf_df.loc[tracks_dict.keys()]
track_array = track_df.values

# transformed_array = model(torch.Tensor(track_array))
# transformed_array = np.dot(track_array, w.T)
transformed_array = np.maximum(0, np.dot(track_array, w.T) + b)
transformed_array = np.dot(transformed_array, w2.T)

track_array = transformed_array #.data.numpy()

In [ ]:
len(track_df)

In [ ]:
def make_query(tracks, drop_track_index, track_df, track_array):
    track_indices = [track_df.index.get_loc(track_uri)
                     for i, track_uri in enumerate(tracks) if i != drop_track_index]
    return np.mean(track_array[track_indices, :], 0)

In [ ]:
# tracks = get_tracks(playlist_list[0])
# make_query(tracks, 0, track_df, track_array).shape

In [ ]:
from sklearn.neighbors import NearestNeighbors
from tqdm import tnrange
import random

def cal_knn(k, track_array, track_df, tracks_dict):
    nbrs = NearestNeighbors(n_neighbors=k, metric='cosine', n_jobs=1).fit(track_array)
    all_true_ids = []
    all_distances = []
    all_suggested_ids = []

    for i in tnrange(10000): #len(playlist_list)):
        tracks = playlist.get_tracks(playlist_list[i])
        if len(tracks) < 5:
            continue
        dropped_track_index = random.choice(range(len(tracks)))
        true_track_uri = tracks[dropped_track_index]
        true_id = tracks_dict[true_track_uri]
        all_true_ids.append(true_id)

        query = make_query(tracks, dropped_track_index, track_df, track_array)
        distances, suggested_index_array = nbrs.kneighbors(query.reshape(1,-1))
        suggested_ids = [tracks_dict[track_df.index[index]]
                         for index in suggested_index_array.flatten()]
        all_distances.append(distances)
        all_suggested_ids.append(suggested_ids)
#         if (i % 1000) == 1:
#             print(i)
#             print(playlist.cal_results(playlist_list, all_true_ids, all_suggested_ids))
    return all_true_ids, all_distances, all_suggested_ids

In [ ]:
all_true_ids, all_distances, all_suggested_ids = cal_knn(500, track_array, track_df, tracks_dict)

In [ ]:
# np.savez('ids_neural_network_50k.npz', all_true_ids=all_true_ids, all_distances=all_distances, all_suggested_ids=all_suggested_ids)

In [ ]:
x=np.load('ids_neural_network_50k.npz')
all_true_ids=x['all_true_ids']
all_suggested_ids=x['all_suggested_ids']

In [ ]:
all_true_ids.shape
all_suggested_ids.

In [ ]:
playlist.cal_results(playlist_list, all_true_ids, all_suggested_ids.tolist())

In [ ]:
results = playlist.cal_results(playlist_list, all_true_ids, all_suggested_ids.tolist())
filename = 'results%d_w2v_neural_networks_enc.txt' % (len(playlist_list),)
with open(filename, 'w') as output:
    output.write(str(results))